In [1]:
# 12.1 Case Study: Predicting Successful Grant Applications
Sys.setlocale("LC_TIME", "English")

library(parallel)
setDefaultCluster(makeCluster(4))

source('CreateGrantData.R')

[1] "English_United States.1252"

Loading required package: lattice
Loading required package: ggplot2

Attaching package: 'lubridate'

The following object is masked from 'package:plyr':

    here

The following object is masked from 'package:base':

    date



 chr [1:1073] "NumCI" "NumDR" "NumECI" "NumEA" "NumHV" "NumPS" "NumSCI" ...
 chr [1:252] "NumCI" "NumDR" "NumECI" "NumPS" "NumSCI" "NumSR" "NumUNK" ...


`there are 1073 predictors in fullSet, However the book show only 1070 predictors @page 311`

In [30]:
reducedCovMat <- cov(training[,reducedSet])
library(subselect)
trimmingResults <- trim.matrix(reducedCovMat)
names(trimmingResults)
trimmingResults$names.discarded

[1] "trimmedmat"        "numbers.discarded" "names.discarded"  
[4] "size"

In [34]:
#apply to full set
fullCovMat <- cov(training[,fullSet])
fullSetResults <- trim.matrix(fullCovMat)
fullSetResults$names.discarded

[1] "NumDR"       "PS.1975"     "CI.Dept1798" "PS.Dept3028" "PS.Faculty1"
[6] "BTotal"      "numPeople"   "Apr"         "Sun"

```r
### Section 12.1 Case Study: Predicting Successful Grant Applications

load("grantData.RData")

library(caret)
library(doMC)
registerDoMC(12)
library(plyr)
library(reshape2)

## Look at two different ways to split and resample the data. A support vector
## machine is used to illustrate the differences. The full set of predictors
## is used. 

pre2008Data <- training[pre2008,]
year2008Data <- rbind(training[-pre2008,], testing)

set.seed(552)
test2008 <- createDataPartition(year2008Data$Class, p = .25)[[1]]

allData <- rbind(pre2008Data, year2008Data[-test2008,])
holdout2008 <- year2008Data[test2008,]

## Use a common tuning grid for both approaches. 
svmrGrid <- expand.grid(sigma = c(.00007, .00009, .0001, .0002),
                        C = 2^(-3:8))

## Evaluate the model using overall 10-fold cross-validation
ctrl0 <- trainControl(method = "cv",
                      summaryFunction = twoClassSummary,
                      classProbs = TRUE)
set.seed(477)
svmFit0 <- train(pre2008Data[,fullSet], pre2008Data$Class,
                 method = "svmRadial",
                 tuneGrid = svmrGrid,
                 preProc = c("center", "scale"),
                 metric = "ROC",
                 trControl = ctrl0)
svmFit0

### Now fit the single 2008 test set
ctrl00 <- trainControl(method = "LGOCV",
                       summaryFunction = twoClassSummary,
                       classProbs = TRUE,
                       index = list(TestSet = 1:nrow(pre2008Data)))


set.seed(476)
svmFit00 <- train(allData[,fullSet], allData$Class,
                  method = "svmRadial",
                  tuneGrid = svmrGrid,
                  preProc = c("center", "scale"),
                  metric = "ROC",
                  trControl = ctrl00)
svmFit00

## Combine the two sets of results and plot

grid0 <- subset(svmFit0$results,  sigma == svmFit0$bestTune$sigma)
grid0$Model <- "10-Fold Cross-Validation"

grid00 <- subset(svmFit00$results,  sigma == svmFit00$bestTune$sigma)
grid00$Model <- "Single 2008 Test Set"

plotData <- rbind(grid00, grid0)

plotData <- plotData[!is.na(plotData$ROC),]
xyplot(ROC ~ C, data = plotData,
       groups = Model,
       type = c("g", "o"),
       scales = list(x = list(log = 2)),
       auto.key = list(columns = 1))

```